### **Лабораторная работа №3**
#### *Выполнил: студент группы 8ПМ11, Кайдаров Алибек*

Вариант 4.

- Получить
    - двадцать пользователей с наибольшим количеством фолловеров;
    - двадцать пользователей с наибольшим количеством друзей;
    - двадцать наиболее часто упоминаемых пользователей.
- Определите топ-5 уникальных хештегов для каждой из выделенных групп.
- Выделить пользователей
    - которые попадают в несколько из этих списков,
    - которые попали только в один список.

Загрузка библиотек и создание новой сессии Spark

In [1]:
from time import time
time_start = time()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, split, explode, lower

In [3]:
spark = SparkSession.builder.appName("Lab_3").master("local[4]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

Создание нового датафрейма на основе датасета *FIFA.csv* и его анализ

In [4]:
dataFrame = spark.read.csv("D:\Spring_2022\DSandCC\\FIFA.csv", inferSchema=True, header=True, multiLine=True, quote='\"', escape='\"')

In [5]:
dataFrame.limit(5)

ID,lang,Date,Source,len,Orig_Tweet,Tweet,Likes,RTs,Hashtags,UserMentionNames,UserMentionID,Name,Place,Followers,Friends
1013597060640145408,en,2018-07-02 01:35:45,Twitter for Android,140,RT @Squawka: Only...,Only two goalkeep...,0,477,"WorldCup,POR,ENG",Squawka Football,Squawka,Cayleb,Accra,861,828
1013597056219295744,en,2018-07-02 01:35:44,Twitter for Android,139,RT @FCBarcelona: ...,scores the winnin...,0,1031,WorldCup,"FC Barcelona,Ivan...","FCBarcelona,ivanr...",Febri Aditya,Bogor,667,686
1013597047482544130,en,2018-07-02 01:35:42,Twitter for Android,107,RT @javierfernand...,Tonight we have b...,0,488,worldcup,"Javier Fernandez,...","javierfernandez,E...",??,null,65,67
1013597044198391808,en,2018-07-02 01:35:41,Twitter Web Client,142,We get stronger T...,We get stronger T...,0,0,"PowerByEXO,WorldC...","EXO,FIFA World Cup ?","weareoneEXO,FIFAW...",Frida Carrillo,"Zapopan, Jalisco",17,89
1013597039999926272,en,2018-07-02 01:35:40,Twitter for Android,140,RT @Squawka: Only...,Only two goalkeep...,0,477,"WorldCup,POR,ENG",Squawka Football,Squawka,tar,null,137,216


In [6]:
dataFrame.printSchema()

root
 |-- ID: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- len: integer (nullable = true)
 |-- Orig_Tweet: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- RTs: integer (nullable = true)
 |-- Hashtags: string (nullable = true)
 |-- UserMentionNames: string (nullable = true)
 |-- UserMentionID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Place: string (nullable = true)
 |-- Followers: integer (nullable = true)
 |-- Friends: integer (nullable = true)



In [7]:
dataFrame.count(), len(dataFrame.columns), 

(530000, 16)

Датасет содержит переменные различных типов, уникальным идентефикатором является переменная *ID*, также есть отсутсвующие данныею.
Описание переменных:
"Lang" - язык твита;
"Date" - когда это было написано в твиттере;
"Source" - устройство/носитель, с которого был сделан твит;
"len" - длина твита;
"Orig_Tweet" - твит в исходном виде;
"Tweet" - обновленный твит после предварительной обработки;
"Likes" - количество лайков, полученных твитом (до момента извлечения);
"RTs" - количество раз, когда твитом поделились;
"Hashtags" - хэштеги, найденные в исходном твите;
"UserMentionNames" и "UserMentionID" - упоминаемые пользователи и их
 имена извлеченные из исходного твита;
"Name" и "Place" - имя и местоположение пользователя;
"Followers" - количество подписчиков, которое есть у учетной записи
пользователя;
"Friends" - количество друзей у учетной записи пользователя.

Для дальнейшего анализа проведена выборка необходимых для задания переменных.

In [8]:
dataFrame = dataFrame.select(["Hashtags", "UserMentionNames", "Name", "Followers", "Friends"])

Некоторые значения датафрейма содержат также имена, состоящие только из вопросительных знаков, не позволяющие с
достаточной достоверностью определить имя пользователя

In [9]:
count_before = dataFrame.count()
dataFrame = dataFrame.filter(col("Name").rlike("[a-zA-Z0-9_ ]"))
count_after = dataFrame.count()
total_missed = count_before - count_after
print(f"В процессе подготовки данных удалено {total_missed} записей")

В процессе подготовки данных удалено 22923 записей


Выборка пользователей с наибольшим количеством фолловеров
Для получения всех твитов от пользователей из выборки проведена операция объединения
датафреймов *dataFrame* и *top_by_followers*

In [10]:
top_by_followers = dataFrame.select(["Name", "Followers"]).groupBy(["Name"])\
    .agg(max(col("Followers")).alias("Max_followers")).sort("Max_followers", ascending=False).limit(20)
top_by_followers = top_by_followers.select("Name")
top_by_followers.show()

+--------------------+
|                Name|
+--------------------+
|                 CNN|
|                ESPN|
|        FC Barcelona|
|    BBC News (World)|
|       The Economist|
|    Reuters Top News|
|   Manchester United|
|      Premier League|
|            Fox News|
|The Wall Street J...|
|      Twitter Sports|
|                9GAG|
|      Vogue Magazine|
|                Xbox|
|The Associated Press|
|            FIFA.com|
|          Chelsea FC|
|   China Xinhua News|
|      Times of India|
|                NDTV|
+--------------------+



Выборка пользователей с наибольшим количеством друзей
Для получения всех твитов от пользователей из выборки проведена операция объединения
датафреймов *dataFrame* и *top_by_friends*

In [11]:
top_by_friends = dataFrame.select(["Name", "Friends"]).groupBy(["Name"])\
    .agg(max(col("Friends")).alias("Max_friends")).sort("Max_friends", ascending=False).limit(20)
top_by_friends = top_by_friends.select("Name")
top_by_friends.show()

+--------------------+
|                Name|
+--------------------+
|                J.D.|
|Footbal, News, Sp...|
|          Tom Hall ?|
|    Michael G Harbut|
|                E.O.|
|              SKYLAR|
|Consumers Federat...|
|        Liverpool FC|
|Real Marsha Wrigh...|
|        Adamu hassan|
|   ?? #PowerWoman ??|
|     ? Yübbî? Umoh ?|
|         Köksal Ak?n|
|         Ben william|
|                 sam|
|   Raphael M. Ny?ng?|
|Zlatan Khubaibimovi?|
|      Tunde Eleniyan|
|     Buffy, Chun-Lit|
|              Futbol|
+--------------------+



Выборка пользователей с наибольшим количеством друзей
Для получения всех твитов от пользователей из выборки проведена операция
объединения датафреймов dataFrame и top_by_mention

In [12]:
top_by_mention = dataFrame.select(split(col("UserMentionNames"), ",").alias("UserMentionNames"))
top_by_mention = top_by_mention.withColumn("UserMentionNames", explode("UserMentionNames").alias("key", "value"))
top_by_mention = top_by_mention.filter(col("UserMentionNames").rlike("[a-zA-Z0-9_ ]"))
top_by_mention = top_by_mention.select("UserMentionNames")\
    .groupBy("UserMentionNames").count().sort("count", ascending=False).limit(20)
top_by_mention = top_by_mention.select(col("UserMentionNames").alias("Name"))
top_by_mention.show()

+------------------+
|              Name|
+------------------+
|  FIFA World Cup ?|
|    FIFA World Cup|
|    Khaled Beydoun|
|               EXO|
|         HNS | CFF|
|        Paul Pogba|
|  Match of the Day|
| Manchester United|
|        Harry Kane|
|      B/R Football|
|       Luka Modri?|
|           England|
|    Premier League|
|              9GAG|
|       French Team|
|             SPORF|
|   Bleacher Report|
|           OptaJoe|
|Belgian Red Devils|
|    Troll Football|
+------------------+



Таким образом получено 3 датафрейма содержащих группы из 20 пользователей и 3 датфрейма
содержащих выборку по именам пользователей соответствующей категории.

Для определения 5 наиболее часто встречающихся хэштегов для каждой категории записана функция *top_hashtags()*.
Функция применена к ранее полученным датафреймам.

In [13]:
def top_hashtags(df):
    temp_frame = dataFrame.join(df, "Name", "right")
    temp_frame = temp_frame.withColumn("Hashtags", lower(col("Hashtags")).alias("Hashtags"))
    temp_frame = temp_frame.select(split(col("Hashtags"), ",").alias("Hashtags"))
    temp_frame = temp_frame.withColumn("Hashtags", explode("Hashtags").alias("key", "value"))
    temp_frame = temp_frame.select("Hashtags").groupBy("Hashtags").count()\
        .sort("count", ascending=False).limit(5)
    temp_frame = temp_frame.select("Hashtags")
    return temp_frame

In [14]:
top_hashtags_by_followers = top_hashtags(top_by_followers)
top_hashtags_by_followers.show()

+--------+
|Hashtags|
+--------+
|worldcup|
|     fra|
|     den|
|     cro|
|  frabel|
+--------+



In [15]:
top_hashtags_by_friends = top_hashtags(top_by_friends)
top_hashtags_by_friends.show()

+---------------+
|       Hashtags|
+---------------+
|       worldcup|
|  worldcupfinal|
|            fra|
|            cro|
|ntvworldcup2018|
+---------------+



In [16]:
top_hashtags_by_mentions = top_hashtags(top_by_mention)
top_hashtags_by_mentions.show()

+--------+
|Hashtags|
+--------+
|worldcup|
|     fra|
|  frabel|
|     cro|
|     bel|
+--------+



Для выполнения последней задачи созданы 3 выборки для каждой группы хэштегов в которых, в дальнейшем, будет проведен
поиск пользователей

In [17]:
hashtag_frame = dataFrame.withColumn("Hashtags", lower(col("Hashtags")).alias("Hashtags"))
hashtag_frame = hashtag_frame.withColumn("Hashtags", split(col("Hashtags"), ",").alias("Hashtags"))
hashtag_frame = hashtag_frame.withColumn("Hashtags", explode("Hashtags").alias("key", "value"))
hashtag_frame_by_followers = hashtag_frame.join(top_hashtags_by_followers, "Hashtags", "inner")\
    .dropna().select("Name").distinct()
hashtag_frame_by_friends = hashtag_frame.join(top_hashtags_by_friends, "Hashtags", "inner")\
    .dropna().select("Name").distinct()
hashtag_frame_by_mention = hashtag_frame.join(top_hashtags_by_mentions, "Hashtags", "inner")\
    .dropna().select("Name").distinct()

In [18]:
not_unique_by_followers = top_by_followers\
    .join(hashtag_frame_by_followers, "Name", "inner")\
    .join(hashtag_frame_by_friends, "Name", "inner")\
    .join(hashtag_frame_by_mention, "Name", "inner").distinct()
unique_by_followers = top_by_followers.join(not_unique_by_followers, "Name", "leftouter")
not_unique_by_friends = top_by_friends\
    .join(hashtag_frame_by_followers, "Name", "inner")\
    .join(hashtag_frame_by_friends, "Name", "inner")\
    .join(hashtag_frame_by_mention, "Name", "inner").distinct()
unique_by_friends = top_by_friends.join(not_unique_by_friends, "Name", "leftouter")
not_unique_by_mention = top_by_mention\
    .join(hashtag_frame_by_followers, "Name", "inner")\
    .join(hashtag_frame_by_friends, "Name", "inner")\
    .join(hashtag_frame_by_mention, "Name", "inner").distinct()
unique_by_mention = top_by_mention.join(not_unique_by_mention, "Name", "leftouter")

In [19]:
not_unique_by_followers.show()

+--------------------+
|                Name|
+--------------------+
|      Premier League|
|   Manchester United|
|The Associated Press|
|      Twitter Sports|
|                ESPN|
|    BBC News (World)|
|          Chelsea FC|
|        FC Barcelona|
|                NDTV|
|   China Xinhua News|
|      Times of India|
|    Reuters Top News|
+--------------------+



In [20]:
not_unique_by_friends.show()

+--------------------+
|                Name|
+--------------------+
|         Ben william|
|   Raphael M. Ny?ng?|
|     Buffy, Chun-Lit|
|    Michael G Harbut|
|        Adamu hassan|
|Real Marsha Wrigh...|
|         Köksal Ak?n|
|Zlatan Khubaibimovi?|
|              Futbol|
|Consumers Federat...|
|          Tom Hall ?|
|                 sam|
|      Tunde Eleniyan|
|   ?? #PowerWoman ??|
+--------------------+



In [21]:
not_unique_by_mention.show()

+------------------+
|              Name|
+------------------+
|  Match of the Day|
|    Premier League|
| Manchester United|
|        Paul Pogba|
|    Troll Football|
|       French Team|
|        Harry Kane|
|       Luka Modri?|
|           OptaJoe|
|         HNS | CFF|
|               EXO|
|             SPORF|
|  FIFA World Cup ?|
|Belgian Red Devils|
|           England|
|      B/R Football|
+------------------+



In [22]:
unique_by_followers.show()

+--------------------+
|                Name|
+--------------------+
|            FIFA.com|
|The Wall Street J...|
|      Premier League|
|   Manchester United|
|The Associated Press|
|      Twitter Sports|
|                ESPN|
|      Vogue Magazine|
|    BBC News (World)|
|          Chelsea FC|
|                9GAG|
|        FC Barcelona|
|                NDTV|
|            Fox News|
|                 CNN|
|                Xbox|
|   China Xinhua News|
|       The Economist|
|      Times of India|
|    Reuters Top News|
+--------------------+



In [23]:
unique_by_friends.show()

+--------------------+
|                Name|
+--------------------+
|                E.O.|
|                J.D.|
|         Ben william|
|   Raphael M. Ny?ng?|
|Footbal, News, Sp...|
|     Buffy, Chun-Lit|
|     ? Yübbî? Umoh ?|
|    Michael G Harbut|
|        Adamu hassan|
|              SKYLAR|
|Real Marsha Wrigh...|
|         Köksal Ak?n|
|Zlatan Khubaibimovi?|
|              Futbol|
|        Liverpool FC|
|Consumers Federat...|
|          Tom Hall ?|
|                 sam|
|      Tunde Eleniyan|
|   ?? #PowerWoman ??|
+--------------------+



In [24]:
unique_by_mention.show()

+------------------+
|              Name|
+------------------+
|  Match of the Day|
|    Premier League|
| Manchester United|
|        Paul Pogba|
|   Bleacher Report|
|    Troll Football|
|       French Team|
|        Harry Kane|
|       Luka Modri?|
|              9GAG|
|           OptaJoe|
|         HNS | CFF|
|               EXO|
|             SPORF|
|  FIFA World Cup ?|
|Belgian Red Devils|
|           England|
|    FIFA World Cup|
|      B/R Football|
|    Khaled Beydoun|
+------------------+



In [25]:
# spark.stop()
print(time() - time_start)

127.19369697570801
